In [3]:
import sys
import cv2
import shutil
sys.path.insert(0, '..')
from utils.tools.dataloader import BKKIterator
from utils.config import cfg
import torch
from utils.config import cfg
from utils.blocks.forecaster import Forecaster
from utils.blocks.encoder import Encoder
from collections import OrderedDict
from utils.blocks.module import EF, Predictor
from torch.optim import lr_scheduler
from utils.loss import Weighted_mse_mae
from utils.blocks.trajGRU import TrajGRU
from utils.train_and_test import train_and_test
import numpy as np
from utils.tools.evaluation import *
from experiment.net_params import *
from utils.tools.visualization import save_hko_movie

ModuleNotFoundError: No module named 'moviepy'

In [4]:
!pip install moviepy

  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Using cached imageio-2.19.2-py3-none-any.whl (3.4 MB)
  Using cached imageio_ffmpeg-0.4.7-py3-none-win_amd64.whl (22.6 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\fact-pc\\anaconda3\\envs\\torch\\lib\\site-packages\\numpy-1.22.3.dist-info\\METADATA'



In [ ]:
!pip uninstall torch

In [ ]:
# encoder = Encoder(encoder_params[0], encoder_params[1]).to(cfg.GLOBAL.DEVICE)
# forecaster = Forecaster(forecaster_params[0], forecaster_params[1])
# encoder_forecaster = EF(encoder, forecaster).to(cfg.GLOBAL.DEVICE)

model = Predictor(conv2d_params).to(cfg.GLOBAL.DEVICE)

model.load_state_dict(torch.load(os.path.join(cfg.GLOBAL.MODEL_SAVE_DIR,'conv2d', 'models', 'encoder_forecaster_20000.pth')))


bkk_iter = BKKIterator(pd_path=cfg.ONM_PD.RAINY_TEST,
                       sample_mode="random",
                       seq_len=IN_LEN + OUT_LEN)

valid_batch, valid_mask, sample_datetimes, _ = bkk_iter.sample(batch_size=1)

valid_batch = valid_batch.astype(np.float32) / 255.0
valid_data = valid_batch[:IN_LEN, ...]
valid_label = valid_batch[IN_LEN:IN_LEN + OUT_LEN, ...]
mask = valid_mask[IN_LEN:IN_LEN + OUT_LEN, ...].astype(int)
torch_valid_data = torch.from_numpy(valid_data).to(cfg.GLOBAL.DEVICE)

with torch.no_grad():
    output = model(torch_valid_data)

output = np.clip(output.cpu().numpy(), 0.0, 1.0)

base_dir = '.'
# S*B*1*H*W
label = valid_label[:, 0, 0, :, :]
output = output[:, 0, 0, :, :]
mask = mask[:, 0, 0, :, :].astype(np.uint8)
save_hko_movie(label, sample_datetimes[0], mask, masked=True,
                       save_path=os.path.join(base_dir, 'ground_truth.mp4'))
save_hko_movie(output, sample_datetimes[0], mask, masked=True,
               save_path=os.path.join(base_dir, 'pred.mp4'))
